In [78]:
import gurobipy as gp
from gurobipy import GRB

# The Model

In [79]:
theater_seat_organizer = gp.Model()

# The Parameters

In [240]:
test_name = "Test Case 3 (exceeding theatre capacity)"
number_of_customers_in_group = [5,6,2,2,3,4,8]
customer_vaccination_status = [ [0,0,1,0,1], [0,1,0,0,0,1], [1, 1], [1, 1], [1, 1, 0], [1, 1, 0, 0], [1,1,1,1,0,0,0,0] ]
number_of_seats = 12
non_vax_cost = 0.01

# Parameter Validation Logic

In [241]:
def isBinary(num):
    if(num == 1 or num == 0):
        return True
    else:
        return False


# Vaccine Records Should Match Customer Records
number_of_vax = count = sum( [ len(customers) for customers in customer_vaccination_status])

if(sum(number_of_customers_in_group) != number_of_vax):
    raise ValueError('Vaccine Records Do Not Match Customer Records')
    
# Vaccine Records Should be Formatted like Customer Records
for index, customer_ofGroup_vaxList in enumerate(customer_vaccination_status):
    if any(not isBinary(person) for person in customer_ofGroup_vaxList):
        raise ValueError('Invalid Arg: Person is either vaxed = 1 or unvaxxed = 0')
        
    if(len(customer_ofGroup_vaxList) != number_of_customers_in_group[index]):
        raise ValueError('Vaccine Records Are Not Formatted Correctly')



# Decision Variables

In [242]:
person_inGroup_atSeat = {}

for group_num in range(len(number_of_customers_in_group)):
    for customer_index in range(0, number_of_customers_in_group[group_num]):
        for seat in range(number_of_seats):
            
            person_inGroup_atSeat[customer_index, group_num, seat] = theater_seat_organizer.addVar(vtype = GRB.BINARY, name = "person" + str(customer_index) + "_inGroup" + str(group_num) + "_atSeat" + str(seat))

groupAccepted = {}
for group_num in range(len(number_of_customers_in_group)):
    groupAccepted[group_num] = theater_seat_organizer.addVar(vtype = GRB.BINARY, name = "group" + str(group_num) + "_admitted")


            

# The Model

In [243]:
objectiveFunction = gp.LinExpr()

for group_num in range(len(number_of_customers_in_group)):
    for customer_index in range(0, number_of_customers_in_group[group_num]):
        for seat in range(number_of_seats):
            objectiveFunction += person_inGroup_atSeat[customer_index, group_num, seat]   
    
for group_num in range(len(number_of_customers_in_group)):
    objectiveFunction -= non_vax_cost * (1-groupAccepted[group_num])

theater_seat_organizer.setObjective(objectiveFunction,  GRB.MAXIMIZE)

# Constraints

In [244]:
# at most one person per seat
for seat in range(number_of_seats):
    at_most_one_person_per_seat = gp.LinExpr()
    
    for group_num in range(len(number_of_customers_in_group)):
        for customer_index in range(0, number_of_customers_in_group[group_num]):
            at_most_one_person_per_seat += person_inGroup_atSeat[customer_index, group_num, seat]   
    theater_seat_organizer.addConstr(at_most_one_person_per_seat <= 1, name = "at_most_one_person_at_seat" + str(seat))



In [245]:
# the whole group must be added
for group_num in range(len(number_of_customers_in_group)):
    
    group_must_be_added = gp.LinExpr() # collects the sum of all the seats the group sits in
    is_group_accepted = gp.LinExpr(number_of_customers_in_group[group_num] * groupAccepted[group_num]) #determines if the group will be seated or not
    
    for customer_index in range(0, number_of_customers_in_group[group_num]):
        for seat in range(number_of_seats):
            group_must_be_added += person_inGroup_atSeat[customer_index, group_num, seat]   
    
    theater_seat_organizer.addConstr(group_must_be_added == is_group_accepted, name = "is_group_" + str(group_num) + "_accepted")


In [246]:
# each person should be assigned to only one seat 
for group_num in range(len(number_of_customers_in_group)):
    for customer_index in range(0, number_of_customers_in_group[group_num]):
        seats_per_person = gp.LinExpr()
        
        for seat in range(number_of_seats):
            seats_per_person += person_inGroup_atSeat[customer_index, group_num, seat] 
            
        theater_seat_organizer.addConstr(seats_per_person <= 1, name = "person" + str(customer_index) + "_group" + str(group_num) + "_uniqueSeat")


In [247]:
# There must be an empty seat between groups 
for seat in range(number_of_seats - 1):
    
    for group_num in range(len(number_of_customers_in_group)):
        for other_group_num in range(len(number_of_customers_in_group)):
            
            if not(group_num == other_group_num):    
                
                for customer_index in range(0, number_of_customers_in_group[group_num]):
                    for customer2_index in range(0, number_of_customers_in_group[other_group_num]):
                        
                        seat1 = person_inGroup_atSeat[customer_index, group_num, seat]
                        seat2 = person_inGroup_atSeat[customer2_index, other_group_num, seat+1]
                        label = "person" + str(customer_index) + "_inGroup" + str(group_num) + "_andPerson" + str(customer2_index) + "+inGroup" + str(other_group_num) + "_cannotSitTogether"
                        theater_seat_organizer.addConstr( seat1 + seat2 <= 1, name = label)

# Optimize 

In [248]:
theater_seat_organizer.optimize()

Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (mac64)
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads
Optimize a model with 341507 rows, 14239 columns and 724115 nonzeros
Model fingerprint: 0xf28300de
Variable types: 0 continuous, 14239 integer (14239 binary)
Coefficient statistics:
  Matrix range     [1e+00, 9e+00]
  Objective range  [1e-02, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]

MIP start from previous solve produced solution with objective -0.07 (0.33s)
MIP start from previous solve produced solution with objective 7.95 (0.33s)
MIP start from previous solve produced solution with objective 8.95 (0.34s)
MIP start from previous solve produced solution with objective 9.95 (0.52s)
MIP start from previous solve produced solution with objective 10.95 (0.68s)
Loaded MIP start from previous solve with objective 10.95
Processed MIP start in 1.61 seconds

Presolve removed 331051 rows and 930 columns
Presolve time: 2.38s
Preso

# Post Process

In [249]:
seating_scheme = list()
for seat in range(number_of_seats):
    seatFound = False
    for group_num in range(len(number_of_customers_in_group)):
        for customer_index in range(0, number_of_customers_in_group[group_num]):
            result = person_inGroup_atSeat[customer_index, group_num, seat]
            
            if( result.x == 1 ):
                seatFound = True
                print( "Seat" + str(seat) + ": person" + str(customer_index) + " fromGroup" + str(group_num))
                
                seating_scheme.append((customer_index, group_num))
    
    if(not seatFound):
        print("Seat" + str(seat) + ": <empty>")
        seating_scheme.append(None)

Seat0: person1 fromGroup0
Seat1: person4 fromGroup0
Seat2: person0 fromGroup0
Seat3: person3 fromGroup0
Seat4: person2 fromGroup0
Seat5: <empty>
Seat6: person2 fromGroup1
Seat7: person4 fromGroup1
Seat8: person0 fromGroup1
Seat9: person5 fromGroup1
Seat10: person1 fromGroup1
Seat11: person3 fromGroup1
[(1, 0), (4, 0), (0, 0), (3, 0), (2, 0), None, (2, 1), (4, 1), (0, 1), (5, 1), (1, 1), (3, 1)]


In [250]:
genericSpacing = list()
last_group = -1
for value in seating_scheme:
    if(type(value) == tuple and value[1] != last_group):
        genericSpacing.append((None,None))
        genericSpacing.append(value)
        last_group = value[1]
        
    
    elif(type(value) == tuple and value[1] == last_group):
        genericSpacing.append(value)
        
genericSpacing.pop(0)

while(genericSpacing[0][1] == genericSpacing[-1][1]):
    genericSpacing.append(genericSpacing[0])
    genericSpacing.pop(0)
    
genericSpacing.pop(0)
    
for members in genericSpacing:
    if not (members[0] == None):
        print("person" + str(members[0]) + " from group " + str(members[1]))
    else:
        print("<SPACE>")



person4 from group 0
person0 from group 0
person3 from group 0
person2 from group 0
<SPACE>
person2 from group 1
person4 from group 1
person0 from group 1
person5 from group 1
person1 from group 1
person3 from group 1


In [251]:
print(groupAccepted)

{0: <gurobi.Var group0_admitted (value 1.0)>, 1: <gurobi.Var group1_admitted (value 1.0)>, 2: <gurobi.Var group2_admitted (value -0.0)>, 3: <gurobi.Var group3_admitted (value -0.0)>, 4: <gurobi.Var group4_admitted (value 0.0)>, 5: <gurobi.Var group5_admitted (value 0.0)>, 6: <gurobi.Var group6_admitted (value 0.0)>}
